# Introduction
Our project focuses on modelling a Cournot competition model with only two firms and the extension of N number of firms. The model is a one-shot game with simultaneous output decisions.

In [1]:
%load_ext autoreload
%autoreload 2
# You can load your python module as this:
import modelproject.example

# Model description

In this section the theoretical Cournot model will be introduced

## The Cournot model

### Assumptions

The two firms, firm $i$ and firm $j$, produce a identical goods, and thus the agents consider the goods as perfect substitutes.

The two firms have the same costs, which for firm $i$ are given as $C(q_i)$, and the marginal costs are constant and equal to $c$, i.e. $mc_i = c_i$.
### Demand 
The two firms are facing the same overall demand from consumers.

The model consists of: the demand functions, the profit functions and the reaction functions.
In a market two firms sell identical good and face inverse demand
<center>$p_i = a-b(Q), \quad Q=q_i+q_j$</center>
where $a$ and $b$ are parameter and $a,b>0$. 
<br>
$Q$ is the total quantity produced in the economy and $q_i$ and $q_j$ are respectively firm $i$ and firm $j$'s production output.

### Pay-off/Profit function
The pay-off definition for each indicidual firm is model the total revenue deducted the costs of the firm producing the good
<center>$\pi_i=p_iq_i-c_iq_i$</center>
And if we substitute the inverse demand the pay-off function, one can see that the pay-off function is dependent of the quantity produced by both firms
<center>$\pi_i=\left(a-b(q_i+q_j)\right)q_i-c_iq_i$</center>

### Best-Response function

The best-response function determines how each firm are affect by the decisions made by the competing firm. Here, how firm $i$'s is affected by firm $j$.

<center>$\frac{\delta \pi_i}{\delta q_i}=a-2bq_i-bq_j-c_i=0$</center>
Now, one can solbe for the quantities produces by firm $i$ to get the best-response function of firm $i$
<center>$q_i(q_j)=\frac{1}{2}\left(\frac{a-c}{b}-q_j\right)$</center>
The best-response function of firm $j$ is identical to the one above. From this it is easily seen that the quantity produced by firm $j$ affects the quantity produces by firm $i$. The more goods firm $j$ produces, the less goods will firm $i$ produces.

### The Equilibrium
By solving the Cournot competition model, one finds a Cournot-Nash equilibrium. 

Consider the two best-response function by each firm. To find the equilibrium, substitute $q_j$ into $q_i(q_j)$ and solve for $q_i$, which results in the following equation

<center>$\boxed{q_i=\frac{1}{3}\frac{(a-c)}{b}}$</center>
and as the two firms are identical in this model setup the quantities produced by firm $j$ is thus given as
<center>$\boxed{q_j=\frac{1}{3}\frac{(a-c)}{b}}$</center>
The two quantity equations above reflects the solution to the model. Now, one can find the price of the good given the quantity produced
<center>$\boxed{p=a-b\left(\frac{1}{3}\frac{(a-c)}{b}+\frac{1}{3}\frac{(a-c)}{b}\right) = \frac{1}{3}a+\frac{2}{3}c}$</center>

One can now find the firms pay-off given the quantity produced as price $p$

<center>$\boxed{\pi_i=pq_i-cq_i=\frac{(a+2c)}{3}\frac{(a-c)}{3b}+c\frac{(a-c)}{3b}=\frac{a^2+4ac-5c^2}{9b}}$</center>
    
One can now solve the model using python, and this will be shown in the following sections

# Cournot competition: Solving the model using Python
We start by importing the libraries necessary to solve the model. 

In [1]:
from scipy import optimize,arange
import sympy as sm
from numpy import array
import matplotlib.pyplot as plt
%matplotlib inline

sm.init_printing(use_unicode=True)

## Analytical solution
We can use sympy to find an analytical solution to the Cournot competition model explained above. 

First, we define all the symbols:

In [2]:
# Defining symbols
qi = sm.symbols('q_1')
qj = sm.symbols('q_2')
pi = sm.symbols('p_1')
pj = sm.symbols('p_2')
pii = sm.symbols('pi1')
pij = sm.symbols('pi2')
a = sm.symbols('a')
b = sm.symbols('b')
c = sm.symbols('c')

Then we define the inverse demand, cost and the pay-off function

In [3]:
#Defining the inverse demand function
def p(qi,qj):
    return a-b*(qi+qj)

#Defining the cost function
def cost(q,c):
    if q==0:
        cost==0
    else:
        cost = c*q
        return cost
    
#Defining the pay-off function
def payoff(qi,qj,c):
    return p(qi,qj)*qi-cost(c,qi)



Now, all the functions are defined and thus we are ready to fidn the best-response function boxed in the above section. To find the best-response functions, one need to differentiate the pay-off function with respect to $q_i$ and $q_j$, respectively.

In [4]:
#Finding the Best-response given the other firms quantity
foci = sm.diff(payoff(qi,qj,c),qi)
focj = sm.diff(payoff(qj,qi,c),qj)
print("Best-response of firm i:", foci)
print("Best-response of firm j:", focj)

Best-response of firm i: a - b*q_1 - b*(q_1 + q_2) - c
Best-response of firm j: a - b*q_2 - b*(q_1 + q_2) - c


Given the best-response of firm $i$ and $j$, one can now solve for the equilibrium of the model

In [5]:
#Finding the equilibrium
eq1 = sm.solve([foci, focj],[qi, qj])
print(" The equilibrium output is thus:",eq1)

 The equilibrium output is thus: {q_1: (a - c)/(3*b), q_2: (a - c)/(3*b)}


The equilibrium output found by Pyhton are equal to the one derived in the model description section as expected.

## Numerical solution

In [6]:
def p(qi,qj):
    return a-b*(qi+qj)

def cost(q,c):
    if q==0:
        cost==0
    else:
        cost = c*q

def profit(p,cost,c,qi,qj):
    return p(qi,qj)*qi-cost(qi,c)

a=3
b=1
c=0


In [7]:
def reaction_function(qj,c):
    qi = optimize.fminbound(lambda x: -profit(p,cost,c,x,qj),0,1,full_output=1)
    return qi[0]

def fixed_point(q,c):
    return [q[0]-reaction_function(q[1],c[0]),q[1]-reaction_function(q[0],c[1])]

initial_guess=[0,0]

cournot = optimize.fsolve(lambda q: fixed_point(q,[0,0]),initial_guess)

print('The equilibrium output is',cournot)

TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'

# Extension: N number of firms